# Question 5

In [38]:
import numpy as np
import pandas as pd
import nltk
import torch
nltk.download('omw-1.4')
import checklist
from copy import deepcopy
from checklist.editor import Editor
from checklist.perturb import Perturb
from sklearn.metrics import precision_score, recall_score
np.random.seed(42)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nedim.azar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
# Loading the subset data
df = pd.read_csv('data/olid-subset-diagnostic-tests.csv')

# Creating a version with typos
df_typos = deepcopy(df)
df_typos['text'] = df['text'].apply(Perturb.add_typos)

df.to_csv('data/no_typos.csv')
df_typos.to_csv('data/typos.csv')

### We got the predictions on Google colab because it wouldn't work locally due to GPU and CUDA issues. We load those predictions in...

In [47]:
predictions = pd.read_csv('predictions/No_Typos_Prediction.csv').drop(['Unnamed: 0'], axis=1)

predictions_typos = pd.read_csv('predictions/Typos_Prediction.csv').drop(['Unnamed: 0'], axis=1)

y_true = predictions['labels']
y_pred = predictions['Predictions']
y_pred_typos = predictions_typos['Predictions']
pos_label = 1

## Precision Scores

In [48]:
# No Typos
precision_score(y_true=y_true, y_pred=y_pred, pos_label=pos_label)

0.9459459459459459

In [27]:
# Typos
precision_score(y_true=y_true, y_pred=y_pred_typos, pos_label=pos_label)

0.9459459459459459

## Recall Scores

In [39]:
# No Typos
recall_score(y_true=y_true, y_pred=y_pred, pos_label=pos_label)

0.7

In [40]:
# Typos
recall_score(y_true=y_true, y_pred=y_pred_typos, pos_label=pos_label)

0.7

## F1 Scores (Macro)

In [43]:
# No Typos
f1_score(y_true=y_true, y_pred=y_pred, pos_label=pos_label, average='macro')

0.8270776116366596

In [44]:
# Typos
f1_score(y_true=y_true, y_pred=y_pred_typos, pos_label=pos_label, average='macro')

0.8270776116366596